In [10]:
from functools import partial
import subprocess
import glob
import time
import numpy as np

TAB_SIZE  = 15
TIMEOUT   = 40
ITER      = 10

kiter_executable = "../Release/bin/kiter"
benchmark_dir = "../benchmarks"

cmdKperiodic = lambda filename : [kiter_executable, "-f", filename,  "-a", "KPeriodicThroughput"]


class BenchmarkResult :
    status_code    = "EMPTY"
    execution_time = None
    stdout         = None

    def __init__ (self, et = None, stdout = None, status_code = "EMPTY") :
        self.status_code = status_code
        self.execution_time = et
        if (type(stdout) == bytes) :
            self.stdout         = stdout.decode("utf-8") 
        else :
            self.stdout         = stdout
            
    def getExecutionTime(self) :
        return self.execution_time
    
    def getComputeTime (self):
        if (not self.stdout) : return None
        for l in self.stdout.split("\n") :
            if "Time" in l :
                return float(l.split(" ")[-1])
    
def runBenchmarkOnce (filename,timeout) :
    
    command = cmdKperiodic(filename = filename)
    
    try :
        start_time = time.time()
        result = subprocess.run(command, stdout=subprocess.PIPE,timeout = timeout)
        end_time = time.time()
        executiontime = (end_time  - start_time) * 1000
        output = result.stdout
        return BenchmarkResult (executiontime  ,  output , "FINISHED")
    except subprocess.TimeoutExpired as e:
        return BenchmarkResult (None  ,  None , "TIMEOUT")
    except FileNotFoundError :
        return BenchmarkResult (None  ,  None , "NOT FOUND")
    except _ as e :
        return BenchmarkResult (None  ,  None , "ERROR")

def runBenchmark (filename, it, timeout) :
    execution_times = []
    compute_times = []
    for _ in range(it) :
        res = runBenchmarkOnce (filename,timeout = timeout)
        execution_times += [res.getExecutionTime()]
        compute_times += [res.getComputeTime()]
    if None in execution_times or None in compute_times :
        return None, None
    return np.median(execution_times) ,  np.median(compute_times)

def formatTime (et , rj = 10) : 
    return '{:.2f} ms'.format(et).rjust(rj) if et else '-'.rjust(rj)
    
def runBenchmarks (filelist, it = ITER, timeout = TIMEOUT) :
    colSize = max([len(x) for x in filelist])

    print("Name".rjust(colSize), "ExecutionTime".rjust(15), "ComputeTime".rjust(15), "Status".rjust(TAB_SIZE))
    for filename in sorted(filelist) :
        execution_time,compute_time  = runBenchmark(filename, it = it, timeout = timeout)
        print(filename.rjust(colSize),
              formatTime(execution_time,TAB_SIZE),
              formatTime(compute_time,TAB_SIZE))


IB5CSDF  = glob.glob(benchmark_dir + '/IB5CSDF/*.xml')
AGB5CSDF = glob.glob(benchmark_dir + '/AGB5CSDF/*.xml')

In [11]:
# This is an example, 
# for correct usage please change it
# - the `timeout` needs to be around 20 sec to execute the entire benchmark.
# - the `it` should be around 10.
runBenchmarks (IB5CSDF, it = 1, timeout = 1) 

                                        Name   ExecutionTime     ComputeTime          Status
      ../benchmarks/IB5CSDF/BlackScholes.xml        19.81 ms         0.74 ms
../benchmarks/IB5CSDF/BlackScholes_sized.xml        12.42 ms         1.04 ms
              ../benchmarks/IB5CSDF/Echo.xml         8.69 ms         0.30 ms
        ../benchmarks/IB5CSDF/Echo_sized.xml               -               -
              ../benchmarks/IB5CSDF/H264.xml       290.36 ms        32.64 ms
        ../benchmarks/IB5CSDF/H264_sized.xml       853.31 ms        70.70 ms
          ../benchmarks/IB5CSDF/JPEG2000.xml        45.66 ms         4.19 ms
    ../benchmarks/IB5CSDF/JPEG2000_sized.xml       887.63 ms       803.34 ms
          ../benchmarks/IB5CSDF/PDectect.xml        27.98 ms         7.64 ms
    ../benchmarks/IB5CSDF/PDectect_sized.xml        37.48 ms        13.02 ms


In [9]:
# This is an example, 
# for correct usage please change it
# - the `timeout` needs to be much higher.
# - the `it` should be around 10.
runBenchmarks (AGB5CSDF, it = 1, timeout = 10) 

                               Name   ExecutionTime     ComputeTime          Status
../benchmarks/AGB5CSDF/autogen1.xml               -               -
../benchmarks/AGB5CSDF/autogen2.xml               -               -
../benchmarks/AGB5CSDF/autogen3.xml               -               -
../benchmarks/AGB5CSDF/autogen4.xml       964.35 ms       113.63 ms
../benchmarks/AGB5CSDF/autogen5.xml      7892.58 ms      6566.93 ms
